In [20]:
import json
import os 
import time
import copernicusmarine
from utils.Copernicus import AdvancedCopernicus as ac
import plotly.graph_objects as go

In [ ]:

ac = ac()   




def get_json_data(file):
    with open(file) as f:
        data = json.load(f)
    return data

marinas = get_json_data('marinas.json')

/Users/marco/Documents/VirtualEnvironments/.main/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
today = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())
one_week_before = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(time.time() - 60*60*24*7))
output_filename = 'subset_output.nc'
 


# find name = Hafen Flensburg Sonwik in marinas.json
def find_marina(name, marinas):
    return [marina for marina in marinas if marina['name'] == name][0]

marina = find_marina('Hafen Flensburg Sonwik', marinas)


marina_name = marina['name']
marina_latitude = marina['location']['latitude']
marina_longitude = marina['location']['longitude']
marina_radius = 0.1

minimum_longitude = marina_longitude - marina_radius
maximum_longitude = marina_longitude + marina_radius

minimum_latitude = marina_latitude - marina_radius
maximum_latitude = marina_latitude + marina_radius


In [14]:
marina_data = ac.get_subset(
    dataset_id="cmems_mod_bal_phy_anfc_PT1H-i",
  dataset_version="202411",
  variables=["thetao", "sla"],
  minimum_longitude=minimum_longitude,
  maximum_longitude=maximum_longitude,
  minimum_latitude=minimum_latitude,
  maximum_latitude=maximum_latitude,
  start_datetime=one_week_before,
  end_datetime=today,
  minimum_depth=0.5016462206840515,
  maximum_depth=0.5016462206840515,
  coordinates_selection_method="strict-inside",
  disable_progress_bar=False,
  output_filename=output_filename
)


INFO - 2025-02-13T09:51:20Z - Selected dataset version: "202411"
INFO - 2025-02-13T09:51:20Z - Selected dataset part: "default"
INFO - 2025-02-13T09:51:24Z - Starting download. Please wait...
100%|██████████| 4/4 [00:00<00:00,  5.98it/s]
INFO - 2025-02-13T09:51:25Z - Successfully downloaded to subset_output.nc


In [24]:
marina_df = marina_data.to_dataframe().reset_index()
marina_df = marina_df.dropna(axis=0, how='any')
marina_df_grouped = marina_df.groupby('time').mean().reset_index()
marina_df_grouped.sort_values(by='time', ascending=False, inplace=True)
marina_df_grouped

,time,depth,latitude,longitude,thetao,sla
167,2025-02-13 10:00:00,0.501646,54.838268,9.49158,2.552219,-0.157219
166,2025-02-13 09:00:00,0.501646,54.838268,9.49158,2.546992,-0.134558
165,2025-02-13 08:00:00,0.501646,54.838268,9.49158,2.550051,-0.135936
164,2025-02-13 07:00:00,0.501646,54.838268,9.49158,2.558844,-0.110666
163,2025-02-13 06:00:00,0.501646,54.838268,9.49158,2.568836,-0.092824
...,...,...,...,...,...,...
4,2025-02-06 15:00:00,0.501646,54.838268,9.49158,3.302244,0.064039
3,2025-02-06 14:00:00,0.501646,54.838268,9.49158,3.302409,0.114216
2,2025-02-06 13:00:00,0.501646,54.838268,9.49158,3.285465,0.171805
1,2025-02-06 12:00:00,0.501646,54.838268,9.49158,3.222528,0.208431


In [26]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=marina_df_grouped['time'],
    y=marina_df_grouped['thetao'],
    mode='lines',
    name='Temperature'
))

fig.update_layout(
    title='Temperature at Hafen Flensburg Sonwik',
    xaxis_title='Time',
    yaxis_title='Temperature'
)

fig.show()

In [13]:
ac.delete_dataset(output_filename)